In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
train_data = np.load('/kaggle/input/exo-data/training.npy')

In [3]:
X_train = train_data[:,:13]
y_train = train_data[:,13:14]

In [4]:
test_data = np.load('/kaggle/input/exo-data/testing.npy')

In [5]:
X_test = test_data[:,:13]
y_test = test_data[:,13:14]

In [7]:
pip install torch_xla

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install neuron

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 39.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.6/144.6 MB 7.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: torch-xla
    Found existing installation: torch-xla 1.0
    Uninstalling torch-xla-1.0:
      Successfully uninstalled torch-xla-1.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.80.0
    Uninstalling google-api-python-client-2.80.0:
      Successfully uninstalled google-api-python-client-2.80.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
earthengine-api 0.1.344 requires google-api-python-client>=1.12.1, but you have google-api-python-client 1.8.0 which is incompatible.


In [12]:
!pip uninstall -y torch
!pip install torch==1.8.2+cpu torchvision==0.9.2+cpu -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html


Found existing installation: torch 1.13.0+cpu
Uninstalling torch-1.13.0+cpu:
  Successfully uninstalled torch-1.13.0+cpu
Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 57.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.0+cpu
    Uninstalling torchvision-0.14.0+cpu:
      Successfully uninstalled torchvision-0.14.0+cpu
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.8.2+cpu which is incompatible.
torchaudio 0.13.0+cpu requires torch==1.13.0, but you have torch 1.8.2+cpu which is incompatible.
pytorch-lightning 1.9.4 requires torch>=1.10.0, but you have torch 1.8

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_xla.core.xla_model as xm

# Define the neural network architecture
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(13, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Move the model to the TPU
device = xm.xla_device()
net = NeuralNet().to(device)

# Prepare the data
# Assuming X_train, y_train, X_test, y_test are already defined and normalized
X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_test = torch.from_numpy(y_test).float().to(device)

# Train the model
epochs = 10
train_loss_list = []
test_loss_list = []
train_r2_list = []
test_r2_list = []

for epoch in range(epochs):
    net.train()
    optimizer.zero_grad()
    y_pred_train = net(X_train)
    train_loss = criterion(y_pred_train, y_train)
    train_loss.backward()
    xm.optimizer_step(optimizer)
    xm.mark_step()

    net.eval()
    with torch.no_grad():
        y_pred_test = net(X_test)
        test_loss = criterion(y_pred_test, y_test)

    train_loss_list.append(train_loss.item())
    test_loss_list.append(test_loss.item())

    train_r2 = 1 - (train_loss.item() / (y_train.var() * len(y_train)))
    test_r2 = 1 - (test_loss.item() / (y_test.var() * len(y_test)))
    train_r2_list.append(train_r2)
    test_r2_list.append(test_r2)

# Plot the loss and R2 scores
import matplotlib.pyplot as plt
plt.plot(train_loss_list, label='Training Loss')
plt.plot(test_loss_list, label='Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(train_r2_list, label='Training R2')
plt.plot(test_r2_list, label='Testing R2')
plt.xlabel('Epochs')
plt.ylabel('R2 Score')
plt.legend()
plt.show()


ImportError: /opt/conda/lib/python3.7/site-packages/_XLAC.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZNK3c1010TensorImpl7stridesEv

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network architecture
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(13, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Move the model and data to the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = NeuralNet().to(device)

X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)
X_test = torch.from_numpy(X_test).float().to(device)
y_test = torch.from_numpy(y_test).float().to(device)

# Train the model
epochs = 100
train_loss_list = []
test_loss_list = []
train_r2_list = []
test_r2_list = []

for epoch in range(epochs):
    net.train()
    optimizer.zero_grad()
    y_pred_train = net(X_train)
    train_loss = criterion(y_pred_train, y_train)
    train_loss.backward()
    optimizer.step()

    net.eval()
    with torch.no_grad():
        y_pred_test = net(X_test)
        test_loss = criterion(y_pred_test, y_test)

    train_loss_list.append(train_loss.item())
    test_loss_list.append(test_loss.item())

    train_r2 = 1 - (train_loss.item() / (y_train.var() * len(y_train)))
    test_r2 = 1 - (test_loss.item() / (y_test.var() * len(y_test)))
    train_r2_list.append(train_r2)
    test_r2_list.append(test_r2)

# Plot the loss and R2 scores
import matplotlib.pyplot as plt
plt.plot(train_loss_list, label='Training Loss')
plt.plot(test_loss_list, label='Testing Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(train_r2_list, label='Training R2')
plt.plot(test_r2_list, label='Testing R2')
plt.xlabel('Epochs')
plt.ylabel('R2 Score')
plt.legend()
plt.show()
